# A brief summary of *Cryptography*
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [*Hashing* Algorithms](#hashing-algorithms)
* [*Encryption*](#encryption)
  * [*Symmetric* Encryption](#symmetric-encryption)
    * [*AES: ECB* mode](#aes-ecb-mode)
    * [*AES: CBC* mode](#aes-cbc-mode)
    * [*AES: CTR* mode](#aes-ctr-mode)


## *Hashing* Algorithms <a class="anchor" id="hashing-algorithms"></a>

A *hashing* algorithm (or, function) maps any number of input bytes to a fixed number of output bytes, called the *hash*, or *digest*.

Hence, by definition, any hashing function is *lossy*.

<img src="../../.img/func-2in-1out.png" width="300" />

A hashing function is meant to posses the following characteristics:
* *Pre-image* resistance
    * Given a hash, it must be practically impossible, without brute-force, to find an input that maps to this hash.
* *Second Pre-Image* resistance
    * Given an input and its corresponding hash, it must be practically impossible, without brute-force, to find another input that maps to this hash.
* *Collision* resistance
    * It must be practically impossible, without brute-force, to find any two inputs that map to the same hash.

*Note:* Using brute-force, a *(Second) Pre-image* attack is of $O(2^n)$ time complexity, while a *Collision* attack is of $O(\sqrt{2^n})$ time complexity, where $n$ is the number of bits in the hash.

Historically, *MD-5* was used, until it was proven not to be *Collision* resistance.

Currently, the recommended hashing function is *SHA-256*.

| *Algorithm* | *Hash Size (bytes)* |
| --- | --- |
| *MD-5* | 16 |
| *SHA-256* | 32 |

In [3]:
import hashlib
sha256_hasher = hashlib.sha256(b'Hello, world.')
sha256_hasher.hexdigest()

'f8c3bf62a9aa3e6fc1619c250e48abe7519373d3edf41be62eb5dc45199af2ef'

In [9]:
import hashlib
sha256_hasher = hashlib.sha256()
sha256_hasher.update(b'Hello,')
sha256_hasher.update(b' world.')
sha256_hasher.hexdigest()

'f8c3bf62a9aa3e6fc1619c250e48abe7519373d3edf41be62eb5dc45199af2ef'

## *Encryption* <a class="anchor" id="encryption"></a>

*Encryption* transforms so-called *plain-text*, into incomprehensible *cipher-text*.

Encryption algorithms are divided into two categorites:
* *Block Cipher* algorithms, operate on blocks (e.g: 16 bytes per block).
* *Stream Cipher* algorithms, operate on bytes.

For block cipher algorithms, when the data length is not *block-aligned*, padding is added. Padding schemes include,

| *Scheme* | `CASE`: 3 bytes of padding are required | `CASE`: 2 bytes of padding are required |
| --- | --- | --- |
| *PKCS7* | `0x03 0x03 0x03` | `0x02 0x02` |
| *ANSI X.923* | `0x00 0x00 0x03` | `0x00 0x02` |

### *Symmetric* Encryption <a class="anchor" id="symmetric-encryption"></a>

*Symmetric* encryption algorithms use the same *key* to encrypt and decrypt.

*AES*, a 16-byte block cipher algorithm, is, currently, the recommended symmetric encryption algorithm.

It has many modes of operation, including,
* *Electronic Code Book (ECB)*,
* *Cipher Block Chaining (CBC)*, and,
* *Counter (CTR)*.

*Note:* *AES* supports key sizes *128*, *192*, and *256* bits.

#### *AES: ECB* mode <a class="anchor" id="aes-ecb-mode"></a>

In *ECB* mode, each block is encrypted *independent* of any other block.

*ECB* mode is not recommended, since:
1. for the same key and plain-text, the cipher-text is always the same, which is not ideal, and,
2. regardless of the key, patterns can be deciphered from the cipher-text.

For example, the following text-in-image is still renderable, even in cipher-text.

<img src="../../.img/aes-ecb-text-in-img.png" width="400" />

In [63]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

# ...
def test_cipher(cipher, key, data):
    
    # ACTION: Instantiate useful objects.
    encryptor = cipher.encryptor()
    decryptor = cipher.decryptor()
    padder = padding.PKCS7(128).padder()
    unpadder = padding.PKCS7(128).unpadder()

    cipher_text = []
    decrypted_text = []

    # ACTION: Encrypt.
    for item in data:
        str_2_bin = item.encode('ascii')
        cipher_text.append(encryptor.update(padder.update(str_2_bin)))
    cipher_text.append(encryptor.update(padder.finalize()))
    cipher_text_hex = [item.hex() for item in cipher_text]

    # ACTION: Decrypt.
    for item in cipher_text:
        decrypted_text.append(unpadder.update(decryptor.update(item)))
    decrypted_text.append(unpadder.finalize())
    decrypted_text_hex = [item.hex() for item in decrypted_text]

    # ACTION: Print summary.
    data_concat = ''.join(data)
    cipher_text_concat = ''.join(cipher_text_hex)
    decrypted_text_concat = ''.join(decrypted_text_hex)
    decrypted_text_ascii = b''.join(decrypted_text).decode('ascii')

    print('Key Size (Bits)               ->    ', len(key_128)*8)
    print('Key                           ->    ', key.hex())
    print('Plain Text (ASCII)            ->    ', data_concat)
    print('Plain Text Length (Bytes)     ->    ', len(data_concat))
    print('Cipher Text (HEX)             ->    ', cipher_text_concat)
    print('Cipher Text Length (Bytes)    ->    ', len(cipher_text_concat)//2)
    print('Decrypted Text (HEX)          ->    ', decrypted_text_concat)
    print('Decrypted Text (ASCII)        ->    ', decrypted_text_ascii)
    print('Decrypted Text Length (Bytes) ->    ', len(decrypted_text_concat)//2)


In [64]:
# Dummy sentence.
data = ["I ", "am very ", "hungry!"]

# Random key, 128 bits in length.
key_128 = os.urandom(16)

In [65]:
# AES, ECB mode.
aes_cipher = Cipher(algorithms.AES(key_128), modes.ECB(), backend=default_backend())

# ...
test_cipher(cipher=aes_cipher, key=key_128, data=data)

Key Size (Bits)               ->     128
Key                           ->     f22a98cdaa8c9e8a9c7c7686ab4fd16c
Plain Text (ASCII)            ->     I am very hungry!
Plain Text Length (Bytes)     ->     17
Cipher Text (HEX)             ->     45112e5c5f9c941d7bb610721c10aad9dc4b3cb5d2b827a8dc5ad455f59e5626
Cipher Text Length (Bytes)    ->     32
Decrypted Text (HEX)          ->     4920616d20766572792068756e67727921
Decrypted Text (ASCII)        ->     I am very hungry!
Decrypted Text Length (Bytes) ->     17


#### *AES: CBC* mode <a class="anchor" id="aes-cbc-mode"></a>

In *CBC* mode, each plain-text block is *XOR'*-ed with the cipher-text of the previous block. The first block is *XOR'*-ed with what's called an *Initialization Vector (IV)*, that's a randomly generated block-sized text, ideally unique per message.

*Note:* The value of the IV used within any message is not confidential.

<img src="../../.img/aes-cbc-visualized.png" width="600" />

In [66]:
# Random IV, AES-block-sized.
iv_aes = os.urandom(16)

# AES, CBC mode.
aes_cipher = Cipher(algorithms.AES(key_128), modes.CBC(initialization_vector=iv_aes), backend=default_backend())

# ...
test_cipher(cipher=aes_cipher, key=key_128, data=data)

Key Size (Bits)               ->     128
Key                           ->     f22a98cdaa8c9e8a9c7c7686ab4fd16c
Plain Text (ASCII)            ->     I am very hungry!
Plain Text Length (Bytes)     ->     17
Cipher Text (HEX)             ->     9026c437f8366e6ceb316eb6fd9df92aa1914efb81fb3cb3133f5ef139f206af
Cipher Text Length (Bytes)    ->     32
Decrypted Text (HEX)          ->     4920616d20766572792068756e67727921
Decrypted Text (ASCII)        ->     I am very hungry!
Decrypted Text Length (Bytes) ->     17


#### *AES: CTR* mode <a class="anchor" id="aes-ctr-mode"></a>

In *CTR* mode, the data is not encrypted. Instead, values from a 16-byte counter are encrypted, generating a *key stream*. Then, the key stream is *XOR*'ed with the plain-text to generate the cipher-text.

Additionally, an *IV* is used, interpretted as the initial offset of the counter value.

*Note:* *AES* in *CTR* mode behaves more like a stream cipher algorithm, and hence, requires no padding.

<img src="../../.img/aes-ctr-visualized.png" width="600" />

In [67]:
# ...
def test_cipher(cipher, key, data):
    
    # ACTION: Instantiate useful objects.
    encryptor = cipher.encryptor()
    decryptor = cipher.decryptor()

    cipher_text = []
    decrypted_text = []

    # ACTION: Encrypt.
    for item in data:
        str_2_bin = item.encode('ascii')
        cipher_text.append(encryptor.update(str_2_bin))
    cipher_text_hex = [item.hex() for item in cipher_text]

    # ACTION: Decrypt.
    for item in cipher_text:
        decrypted_text.append(decryptor.update(item))
    decrypted_text_hex = [item.hex() for item in decrypted_text]

    # ACTION: Print summary.
    data_concat = ''.join(data)
    cipher_text_concat = ''.join(cipher_text_hex)
    decrypted_text_concat = ''.join(decrypted_text_hex)
    decrypted_text_ascii = b''.join(decrypted_text).decode('ascii')

    print('Key Size (Bits)               ->    ', len(key_128)*8)
    print('Key                           ->    ', key.hex())
    print('Plain Text (ASCII)            ->    ', data_concat)
    print('Plain Text Length (Bytes)     ->    ', len(data_concat))
    print('Cipher Text (HEX)             ->    ', cipher_text_concat)
    print('Cipher Text Length (Bytes)    ->    ', len(cipher_text_concat)//2)
    print('Decrypted Text (HEX)          ->    ', decrypted_text_concat)
    print('Decrypted Text (ASCII)        ->    ', decrypted_text_ascii)
    print('Decrypted Text Length (Bytes) ->    ', len(decrypted_text_concat)//2)


In [72]:
# Random IV, AES-block-sized.
iv_aes = os.urandom(16)

# AES, CTR mode.
aes_cipher = Cipher(algorithms.AES(key_128), modes.CTR(nonce=iv_aes), backend=default_backend())

# ...
test_cipher(cipher=aes_cipher, key=key_128, data=data)

Key Size (Bits)               ->     128
Key                           ->     f22a98cdaa8c9e8a9c7c7686ab4fd16c
Plain Text (ASCII)            ->     I am very hungry!
Plain Text Length (Bytes)     ->     17
Cipher Text (HEX)             ->     4f4d7c83105584c1fe0b9347dce4eb911b
Cipher Text Length (Bytes)    ->     17
Decrypted Text (HEX)          ->     4920616d20766572792068756e67727921
Decrypted Text (ASCII)        ->     I am very hungry!
Decrypted Text Length (Bytes) ->     17
